# **Lab Session 11 - Writing clean and readable code**

- Commenting
- Type hinting
- Pythonic expression (list comprehension, zip, sorted...)
- Bad practices

In [ ]:
# ==== Environment Setup ====
# Detects Colab vs local and provides cross-platform utilities

import os
import sys

# Detect environment
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("✓ Running on Google Colab")
else:
    print("✓ Running locally")

def download_file(url: str, filename: str) -> str:
    """Download file if it doesn't exist. Works on both Colab and local."""
    if os.path.exists(filename):
        print(f"✓ {filename} already exists")
        return filename
    
    print(f"Downloading {filename}...")
    if IN_COLAB:
        import subprocess
        subprocess.run(['wget', '-q', url, '-O', filename], check=True)
    else:
        import urllib.request
        urllib.request.urlretrieve(url, filename)
    print(f"✓ Downloaded {filename}")
    return filename

In [ ]:
# ==== Device Setup ====
import torch

def get_device():
    """Get best available device: CUDA > MPS > CPU."""
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"✓ Using CUDA GPU: {torch.cuda.get_device_name(0)}")
    elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        device = torch.device('mps')
        print("✓ Using Apple MPS (Metal)")
    else:
        device = torch.device('cpu')
        print("✓ Using CPU")
    return device

DEVICE = get_device()

#



In [ ]:
from typing import List

# standard commenting at the beginning of a function
def fahrenheit_to_celsius(f):
    """
    Convert temperature from Fahrenheit to Celsius.

    Input:
        f (float): Temperature in Fahrenheit.

    Output:
        float: Temperature in Celsius.
    """
    return (f - 32) * 5/9


# type hinting with multiple types
def to_str(value: int | float) -> str:
    return str(value)


def to_int(value: str="0") -> int:
    return int(value)

def average(values: list[float]) -> float:
    return sum(values) / len(values)


#TODO Add type hint to the function. Which data types would work with this function? Which wouldn't?


In [ ]:
def meters_to_miles(**kwargs) -> float:
  """Function that transform an input number representing
  metric systems into miles.

  Input:
    kwargs(dict): metric representation of an input number
                  e.g. km=1, m=23, cm=1
  Output:
    float: the distance in miles
  """

  metric_distances = {"km": 10**3,
                      "cm": 10**-2,
                      "mm": 10**-3}

  # list comprehension: increased readability
  # dict.get(key) safer than dict[key]
  meters = sum([v * metric_distances.get(k, 0) for k, v in kwargs.items()])
  miles = meters * 0.000621371
  return miles


  def best_conversion_rate(euro_prices:list, dollar_prices:list) -> float:
    """Suppose you have EUR and want USD. Given a list of prices
    in € and prices in $, finds the best (higher)
    conversion rate, returning its value.

    Input:
      euro_prices[list]: the original prices in €
      dollar_prices[list]: the converted prices in $
    Output:
      float: the best conversion rates
    """
    # zipping:
    price_tuples = list(zip(euro_prices, dollar_prices))
    # sorted:
    sorted_rates = sorted(conversion,
                          key=lambda x: x[1]/x[0],
                          reverse=True)
    best, *rest = sorted_rates
    return best



miles = meters_to_miles(km=10, m=23, dm=100)


In [ ]:
coefficients = {"feet": 30, "inches": 2.54}
new_coefficients = {"feet": 30.48, "inches": 2.54, "hand": 10.16}

# bad practices: modifying mutable input inplace,
# changing dictionary size while looping over its items
# TODO: improve this function
def add_conversion_coefficients(old_coefficients:dict, new_coefficients:dict) -> dict:
  for k, val in new_coefficients:
    if k in old_coefficients:
      old_coefficients.pop(k)
      old_coefficients[k] = val
    else:
      old_coefficients[k] = val
  return old_coefficients

# **Modularization**
- Importance of __init__.py
- Private/protected/public methods
- Decorators (log calls, caching)



In [ ]:
from __future__ import annotations

from functools import wraps
from typing import Callable, Dict, Tuple, Any
from collections import OrderedDict
import math
import time

def log_call(func: Callable) -> Callable:
    """Logging decorator.
    - Checks instance attribute `enable_logs` if available.
    - Logs only function name, args, kwargs, and result.
    """

    @wraps(func)
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        # Derive a friendly function signature string
        logging_enabled = getattr(args[0], "enable_logs", True)

        if logging_enabled:
          arg_list = []
          for a in args[1:] if len(args) and hasattr(args[0], "__class__") else args:
              arg_list.append(repr(a))
          arg_list += [f"{k}={v!r}" for k, v in kwargs.items()]
          signature = ", ".join(arg_list)

          print(f"[LOG] Calling {func.__name__}({signature})")
        else:
          pass

        result = func(*args, **kwargs)
        return result

    return wrapper



def cache(maxsize: int = 128):
    """
    A flexible LRU cache decorator that works with
    any hashable object as a cache key.
    Uses OrderedDict for LRU eviction.
    """
    def decorator(func: Callable):
        cache = OrderedDict()

        @wraps(func)
        def wrapper(*args, **kwargs) -> Any:
            # Build a hashable cache key
            if not kwargs:
                # Fast path: only args
                if len(args) == 1:
                    key = args[0]                    # simplest possible key
                else:
                    key = args                       # tuple of args
            else:
                # Full path: args + sorted kwargs
                key = (args, tuple(sorted(kwargs.items())))

            # Cache hit → move to end
            if key in cache:
                cache.move_to_end(key)
                return cache[key]

            # Cache miss
            result = func(*args, **kwargs)
            cache[key] = result
            cache.move_to_end(key)

            # Enforce maxsize with LRU eviction
            if len(cache) > maxsize:
                cache.popitem(last=False)

            return result

        return wrapper

    return decorator




@cache(10)
def simulate_losing_time(t:int) -> float:
  time.sleep(5)
  return t

t0 = time.time()
res0 = simulate_losing_time(3)
t1 = time.time()
res1 = simulate_losing_time(3)
t2 = time.time()
print(f"Calling function takes {t1-t0:.1f}s the 1st time, {t2-t1:.1f}s the 2nd time.")


Calling function takes 5.0s the 1st time, 0.0s the 2nd time.


In [1]:

class Imperial2Metric:
    """Converter for imperial <-> metric units.

    This class uses public conversion methods (for length, mass, temperature,
    and volume), as well

    Attributes
    ----------
    default_precision: int
        Default number of decimal places to round results to.

    _unit_aliases: Dict[str, str]
        Protected mapping used for alias resolution.
    """

    # Protected class-level dictionary of full abbreviation names
    _unit_aliases: Dict[str, str] = {
        "in": "inch",
        "ft": "foot",
        "yd": "yard",
        "lb": "pound",
        "oz": "ounce",
        "mi": "mile",
        "gal": "gallon",
    }

    def __init__(self, enable_logs:bool = True, default_precision: int = 4) -> None:
        """Initialize the Converter instance.

        Parameters
        ----------
        default_precision: int
            Number of decimal places for rounding conversion results.
        """

        self.enable_logs: bool = enable_logs
        self.default_precision: int = default_precision

    # dunder class (double underscore)
    def __str__(self):
      string = ("Converter for imperial <-> metric units.\nSupports following units: " +
            ", ".join(self._unit_aliases.values()))
      return string


    # Protected methods: convention for methods for internal use (not enforced)
    # I.e. method should not be called by an initialized instance
    def _to_float(self, value: Any) -> float:
        """Protected helper: try to coerce a numeric-like value to float.

        Raises ValueError if coercion fails. This is a small helper used by
        conversion methods to accept ints, floats, and strings like '12.4'.
        """
        try:
            return float(value)
        except (TypeError, ValueError) as exc:
            raise ValueError(f"Value {value!r} is not numeric") from exc


    def _round(self, value: float, precision: int | None = None) -> float:
        """Protected helper: round a value using the instance precision.

        If `precision` is not provided, uses `self.default_precision`.
        """
        if precision is None:
            precision = self.default_precision
        # Use Python's built-in round; keep as float
        return round(value, precision)

    def __validate_unit(self, unit: str, allowed: Tuple[str, ...]) -> str:
        """Private helper: normalizes and validates a unit string.

        This demonstrates a private method that other public methods may call
        to centralize unit validation logic.
        """
        if not isinstance(unit, str):
            raise TypeError("unit must be a string")

        unit_norm = unit.strip().lower()

        # Resolve aliases from protected mapping
        if unit_norm in self._unit_aliases:
            unit_norm = self._unit_aliases[unit_norm]

        if unit_norm not in allowed:
            raise ValueError(f"Unit '{unit}' is not one of {allowed}")

        return unit_norm

    # ---------------------- Public conversion methods --------------------

    @log_call
    @cache
    def length_inches_to_centimeters(self, inches: float, precision: int | None = None) -> float:
        """Convert inches to centimeters.

        This method demonstrates:
         - public API
         - use of both decorators (note order: simple_cache runs first)
         - type hints
        """
        inches_f = self._to_float(inches)
        cm = inches_f * 2.54
        return self._round(cm, precision)

    @log_call
    @cache
    def length_feet_to_meters(self, feet: float, precision: int | None = None) -> float:
        """Convert feet to meters.

        1 foot = 0.3048 meters.
        """
        feet_f = self._to_float(feet)
        meters = feet_f * 0.3048
        return self._round(meters, precision)

    @log_call
    @cache
    def mass_pounds_to_kilograms(self, pounds: float, precision: int | None = None) -> float:
        """Convert pounds (lb) to kilograms (kg).

        1 pound = 0.45359237 kilograms.
        """
        lb_f = self._to_float(pounds)
        kg = lb_f * 0.45359237
        return self._round(kg, precision)

    @log_call
    @cache
    def volume_gallons_to_liters(self, gallons: float, precision: int | None = None) -> float:
        """Convert US liquid gallons to liters.

        1 US gallon = 3.785411784 liters.
        """
        gal_f = self._to_float(gallons)
        liters = gal_f * 3.785411784
        return self._round(liters, precision)

    @log_call
    def temperature_fahrenheit_to_celsius(self, fahrenheit: float, precision: int | None = None) -> float:
        """Convert Fahrenheit to Celsius.

        Formula: (F - 32) * 5/9
        Note: Not decorated with cache - demonstrates decorator choice.
        """
        f = self._to_float(fahrenheit)
        c = (f - 32.0) * (5.0 / 9.0)
        return self._round(c, precision)

    # ---------------------- Methods that accept a unit -------------------

    @log_call
    def convert_length(self, value: float, src_unit: str, dst_unit: str, precision: int | None = None) -> float:
        """Convert length between a restricted set of units.

        Supported units (case-insensitive): 'inch', 'foot', 'yard', 'meter', 'centimeter', 'kilometer', 'mile'

        This method demonstrates use of the private __validate_unit helper.
        """
        allowed = ("inch", "foot", "yard", "meter", "centimeter", "kilometer", "mile")
        src = self.__validate_unit(src_unit, allowed)
        dst = self.__validate_unit(dst_unit, allowed)

        # First normalize to meters, then to destination
        value_f = self._to_float(value)
        meters = self._length_to_meters(value_f, src)
        out = self._meters_to_length(meters, dst)
        return self._round(out, precision)

    # Protected metods

    def _length_to_meters(self, value: float, unit: str) -> float:
        """Protected class which converts a length in `unit` to meters.
        """
        unit = unit.lower()
        v = self._to_float(value)

        if unit == "inch":
            return v * 0.0254
        if unit == "foot":
            return v * 0.3048
        if unit == "yard":
            return v * 0.9144
        if unit == "meter":
            return v
        if unit == "centimeter":
            return v / 100.0
        if unit == "kilometer":
            return v * 1000.0
        if unit == "mile":
            return v * 1609.344

        raise ValueError(f"Unsupported length unit: {unit}")

    def _meters_to_length(self, meters: float, unit: str) -> float:
        """Protected class which convert meters to the requested `unit`.
        """
        m = self._to_float(meters)
        unit = unit.lower()

        if unit == "inch":
            return m / 0.0254
        if unit == "foot":
            return m / 0.3048
        if unit == "yard":
            return m / 0.9144
        if unit == "meter":
            return m
        if unit == "centimeter":
            return m * 100.0
        if unit == "kilometer":
            return m / 1000.0
        if unit == "mile":
            return m / 1609.344

        raise ValueError(f"Unsupported length unit: {unit}")



# Test the function

if __name__ == "__main__":
    # Basic demo showing how the class behaves when run as a script.
    conv = Imperial2Metric(default_precision=3, enable_logs=False)

    print("12 in -> cm:", conv.length_inches_to_centimeters(12))
    print("3 ft -> m:", conv.length_feet_to_meters(3))
    print("200 lb -> kg:", conv.mass_pounds_to_kilograms(200))
    print("2 gal -> L:", conv.volume_gallons_to_liters(2))
    print("98.6 F -> C:", conv.temperature_fahrenheit_to_celsius(98.6))

    # convert_length demonstrates unit validation and multi-step conversion
    print("5 mile -> km:", conv.convert_length(5, "mile", "kilometer", precision=5))


NameError: name 'Dict' is not defined

In [ ]:
str(conv)

'Converter for imperial <-> metric units. Supports following units:inch foot yard pound ounce mile gallon'

# **GitHub and project documentation**

- README.md, .gitignore
- Mastering main git cmd, pip, black